![MuJoCo banner](https://raw.githubusercontent.com/google-deepmind/mujoco/main/banner.png)







### Copyright notice

> <p><small><small>Copyright 2025 DeepMind Technologies Limited.</small></p>
> <p><small><small>Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0">http://www.apache.org/licenses/LICENSE-2.0</a>.</small></small></p>
> <p><small><small>Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.</small></small></p>

# Tutorial

This notebook is the first of two that demonstrate pixels-to-actions training using the experimental Madrona rendering backend. 

### Usage

Please note that this Colab **does not support a hosted runtime**. To use it, please install [Madrona-MJX](https://github.com/shacklettbp/madrona_mjx/tree/main) and Playground on your local device and select `Connect to a local runtime`, or download the notebook to run locally! We recommend a device with at least 24 GB VRAM, such as the RTX4090.

In [ ]:
# @title Import MuJoCo, MJX, and Brax
from datetime import datetime
import functools
import os
import time

from brax.training.agents.ppo import networks_vision as ppo_networks_vision
from brax.training.agents.ppo import train as ppo
from IPython.display import clear_output
import jax
from jax import numpy as jp
from matplotlib import pyplot as plt
import mediapy as media
import numpy as np

from mujoco_playground import wrapper

np.set_printoptions(precision=3, suppress=True, linewidth=100)

## Introducing Madrona MJX

[Madrona MJX](https://github.com/shacklettbp/madrona_mjx/tree/main) is Mujoco's experimental on-device rendering backend, developed in collaboration with [Madrona](https://github.com/shacklettbp/madrona). It implements a JAX-compatible batched renderer for rolling out multiple training environments in parallel.

*The Rendering Bottleneck*

In Reinforcement Learning, an agent uses a policy to produce an action $a_t$ in response to an observation $o_t$ of the simulation state. The environment evaluates the interaction via a reward signal $r_t$, then transitions to the next state rendered as $o_{t+1}$. Together, this forms the [transition](https://github.com/google/brax/blob/69637a359463738140c1b850f61ad0088a23538b/brax/training/types.py#L43) $(o_t, a_t, r_t, o_{t+1})$, the basic data unit of a RL training pipeline. Due to poor sample efficiency, the speed of collecting transitions is the often the bottle-neck in on-policy RL methods such as PPO. In addition to full-body physics simulation - for example `mujoco.mjx.step` - pixels-based training requires an expensive call to a renderer on every transition. Along with the cost of expensive CNN architectures, rendering speed becomes a significant performance consideration.

*Coming Up*

In this tutorial, we'll use a high-performance ray-tracing backend to render images in parallel across a batch of environments. Before we get to sub-minute pixel-based training, let's see how much faster Madrona's batched rendering lets us collect raw transitions. For a baseline, let's load the Cartpole Task from dm_control, a popular reference library built on Mujoco:

Now, let's load Mujoco Playground's JAX Cartpole re-implementation. While we're at it, we'll configure it for the upcoming training.

In [ ]:
from mujoco_playground import dm_control_suite

N = 1000
img_shape = (64, 64)
num_envs = 1024
ctrl_dt = 0.04
episode_length = int(3 / ctrl_dt)

config_overrides = {
    "vision": True,
    "vision_config.render_batch_size": num_envs,
    "action_repeat": 1,
    "ctrl_dt": ctrl_dt,
    "episode_length": episode_length,
}

env_name = "CartpoleBalance"
env = dm_control_suite.load(
    env_name, config_overrides=config_overrides
)

env = wrapper.wrap_for_brax_training(
    env,
    vision=True,
    num_vision_envs=num_envs,
    action_repeat=1,
    episode_length=episode_length,
)

In [ ]:
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step)

In [ ]:
key_reset, key_act = jax.random.split(jax.random.PRNGKey(0))
state = jit_reset(jax.random.split(key_reset, num_envs))

# Pre-compile
jit_step = jit_step.lower(
    state, jp.zeros((num_envs, env.action_size))
).compile()

t0 = time.time()
for i in range(N):
  act = jax.random.uniform(
      key_act, (num_envs, env.action_size), minval=-1.0, maxval=1.0
  )
  state = jit_step(state, act)

jax.tree_util.tree_map(
    lambda x: x.block_until_ready(), state
)  # Await device completion
dt = time.time() - t0

print("Madrona MJX: {:d} transitions per second".format(int(N * num_envs / dt)))

## Balancing a Cartpole

Now, let's train a pixels-to-torque policy to balance a cartpole. In [our implementation]((https://github.com/kevinzakka/mujoco_playground/blob/main/mujoco_playground/_src/dm_control_suite/cartpole.py)), we control the observation dimensionality while encouraging the agent to understand dynamics by setting the observations as grayscale images stacked across sequential timesteps.

#### Visualize the environment

In [ ]:
state = jit_reset(jax.random.split(jax.random.PRNGKey(0), num_envs))
rollout = [state]

f = 0.2
for i in range(episode_length):
  t = state.data.time
  action = []
  for j in range(env.action_size):
    action.append(0.0)
  action = jp.tile(jp.array(action), (num_envs, 1))
  state = jit_step(state, action)
  rollout.append(state)

print(rollout[0].obs["pixels/view_0"].shape)
obs = [r.obs["pixels/view_0"][0, ...] for r in rollout]

media.show_video(obs, fps=1.0 / env.dt, height=256)

In [ ]:
plt.imshow(obs[50])
plt.axis("off")
plt.tight_layout()
plt.show()

#### Train
Training the policy takes 57s on a RTX4090 GPU.

In [ ]:
from mujoco_playground.config import dm_control_suite_params

# Load vision-specific PPO configuration tuned for CartpoleBalance
ppo_params = dm_control_suite_params.brax_vision_ppo_config(env_name)
ppo_params.episode_length = episode_length
ppo_params.network_factory = ppo_networks_vision.make_ppo_networks_vision

In [ ]:
x_data, y_data, y_dataerr = [], [], []
times = [datetime.now()]


def progress(num_steps, metrics):
  clear_output(wait=True)

  times.append(datetime.now())
  x_data.append(num_steps)
  y_data.append(metrics["eval/episode_reward"])
  y_dataerr.append(metrics["eval/episode_reward_std"])

  plt.xlim([0, ppo_params["num_timesteps"] * 1.25])
  plt.ylim([0, 100])
  plt.xlabel("# environment steps")
  plt.ylabel("reward per episode")
  plt.title(f"y={y_data[-1]:.3f}")
  plt.errorbar(x_data, y_data, yerr=y_dataerr, color="blue")

  display(plt.gcf())


train_fn = functools.partial(
    ppo.train, **dict(ppo_params), progress_fn=progress
)

In [ ]:
make_inference_fn, params, metrics = train_fn(environment=env)
print(f"time to jit: {times[1] - times[0]}")
print(f"time to train: {times[-1] - times[1]}")

#### Visualize the Policy

In [ ]:
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step)
jit_inference_fn = jax.jit(make_inference_fn(params, deterministic=True))

In [ ]:
rng = jax.random.PRNGKey(42)
rollout = []
n_episodes = 1


def first_env(x):
  return jax.tree.map(
      lambda y: y[0] if hasattr(y, "ndim") and y.ndim > 0 and y.shape[0] == num_envs else y,
      x,
  )


for _ in range(n_episodes):
  reset_rng, rng = jax.random.split(rng)
  state = jit_reset(jax.random.split(reset_rng, num_envs))
  rollout.append(first_env(state))

  for _ in range(episode_length):
    policy_rng, rng = jax.random.split(rng)
    ctrl, _ = jit_inference_fn(state.obs, jax.random.split(policy_rng, num_envs))
    state = jit_step(state, ctrl)
    rollout.append(first_env(state))

render_every = 1
vis_rollout = rollout[::render_every]
frames = env.render(vis_rollout, camera="fixed")

# Visualize the learned observation stream as valid HxWx3 frames.
k = next(iter(vis_rollout[0].obs.keys()))
obs_frames = []
for s in vis_rollout:
  img = np.asarray(s.obs[k])

  # Convert to uint8 in [0, 255].
  if img.dtype != np.uint8:
    img = img.astype(np.float32)
    if np.max(img) <= 1.0:
      img = img * 255.0
    img = np.clip(img, 0, 255).astype(np.uint8)

  # Ensure mediapy-compatible shape HxWx3.
  if img.ndim == 2:
    img = np.repeat(img[..., None], 3, axis=-1)
  elif img.ndim == 3 and img.shape[-1] == 1:
    img = np.repeat(img, 3, axis=-1)
  elif img.ndim == 3 and img.shape[-1] == 3:
    pass
  else:
    raise ValueError(f"Unexpected obs frame shape: {img.shape}")

  obs_frames.append(img)

rewards = np.asarray([float(s.reward) for s in rollout])
media.show_videos([frames, obs_frames], fps=1.0 / env.dt / render_every)

plt.plot(np.convolve(rewards, np.ones(10) / 10, mode="valid"))
plt.xlabel("time step")
plt.ylabel("reward")
plt.show()

🙌 See you in Part 2 of pixels-based training with Madrona-MJX, where we'll cover how to use visual domain randomization to transfer policies to the real world!